Data Transformation

In [8]:
import pandas as pd

# Load the Excel file
file_path = '/content/Solar_Generation_Outputs_2023.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)

# Initialize variables
data_frames = []
current_installation = None
headers = ['Year', 'Size', 'Hour', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Process each row
for index, row in df.iterrows():
    # Check for installation header row
    if 'Adjusted' in str(row[0]):
        current_installation = row[0]
        continue

    # Check for data header row
    if row[0] == 'Year' and row[1] == 'Size':
        column_names = row.values
        data_start = index + 1
        continue

    # Process data rows for current installation
    if current_installation and index >= data_start:
        # Skip empty rows
        if pd.isna(row[0]):
            continue

        # Create a dataframe for this block of data
        data_end = index
        while data_end < len(df) and not pd.isna(df.iloc[data_end, 0]):
            data_end += 1

        installation_data = df.iloc[index:data_end].copy()
        installation_data.columns = column_names
        installation_data['Installation'] = current_installation
        data_frames.append(installation_data)

        # Skip ahead to next installation
        index = data_end - 1

# Combine all data
combined_df = pd.concat(data_frames)

# Melt the dataframe to make it horizontal (long format)
melted = pd.melt(combined_df,
                id_vars=['Installation', 'Year', 'Size', 'Hour'],
                value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                var_name='Month',
                value_name='Generation')

# Convert month names to numbers
month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
             'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
melted['Month'] = melted['Month'].map(month_map)

# Reorder columns
final_df = melted[['Installation', 'Year', 'Month', 'Hour', 'Size', 'Generation']]

# Save to new Excel file
output_path = '/content/Solar_Generation_Outputs_2023_Transformed.xlsx'
final_df.to_excel(output_path, index=False)

print("Data transformation complete. Saved to:", output_path)
final_df.head()

Data transformation complete. Saved to: /content/Solar_Generation_Outputs_2023_Transformed.xlsx
Final shape: (1440, 6)


,Installation,Year,Month,Hour,Size,Generation
0,Total_Tower2_Adjusted_100KWp,2023,1,0,100,0
1,Total_Tower2_Adjusted_100KWp,2023,1,1,100,0
2,Total_Tower2_Adjusted_100KWp,2023,1,2,100,0
3,Total_Tower2_Adjusted_100KWp,2023,1,3,100,0
4,Total_Tower2_Adjusted_100KWp,2023,1,4,100,0


In [10]:
import pandas as pd

# Load the Excel file
file_path = '/content/Solar_Generation_Outputs_2024.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)

# Initialize variables
data_frames = []
current_installation = None
headers = ['Year', 'Size', 'Hour', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
last_processed_index = -1  # Track the last processed row

# Process each row
for index, row in df.iterrows():
    # Skip rows we've already processed
    if index <= last_processed_index:
        continue

    # Check for installation header row
    if isinstance(row[0], str) and 'Adjusted' in row[0]:
        current_installation = row[0]
        continue

    # Check for data header row
    if isinstance(row[0], str) and row[0] == 'Year' and row[1] == 'Size' and row[2] == 'Hour':
        column_names = row.values
        data_start = index + 1
        continue

    # Process data rows for current installation
    if current_installation and index >= data_start:
        # Skip empty rows
        if pd.isna(row[0]):
            continue

        # Find the end of this data block
        data_end = index
        while data_end < len(df) and not pd.isna(df.iloc[data_end, 0]):
            data_end += 1

        # Extract the data block
        installation_data = df.iloc[index:data_end].copy()
        installation_data.columns = column_names
        installation_data['Installation'] = current_installation
        data_frames.append(installation_data)

        # Update the last processed index
        last_processed_index = data_end - 1

# Combine all data
if data_frames:
    combined_df = pd.concat(data_frames)

    # Drop duplicate rows based on all columns
    combined_df = combined_df.drop_duplicates()

    # Melt the dataframe to make it horizontal (long format)
    month_cols = [col for col in combined_df.columns if col in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

    melted = pd.melt(combined_df,
                    id_vars=['Installation', 'Year', 'Size', 'Hour'],
                    value_vars=month_cols,
                    var_name='Month',
                    value_name='Generation')
    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    melted['Month'] = melted['Month'].map(month_map)
    final_df = melted[['Installation', 'Year', 'Month', 'Hour', 'Size', 'Generation']]
    final_df = final_df.drop_duplicates(subset=['Installation', 'Year', 'Month', 'Hour'])
    output_path = '/content/Solar_Generation_Outputs_2024_Transformed.xlsx'
    final_df.to_excel(output_path, index=False)

    print("Data transformation complete. Saved to:", output_path)
    print("\nFirst 5 rows of transformed data:")
    print(final_df.head())
    print("\nShape of final dataframe:", final_df.shape)
else:
    print("No valid data found in the Excel file.")

Data transformation complete. Saved to: /content/Solar_Generation_Outputs_2024_Transformed.xlsx

First 5 rows of transformed data:
                   Installation  Year  Month Hour Size Generation
0  Total_Tower2_Adjusted_100KWp  2024      1    0  100          0
1  Total_Tower2_Adjusted_100KWp  2024      1    1  100          0
2  Total_Tower2_Adjusted_100KWp  2024      1    2  100          0
3  Total_Tower2_Adjusted_100KWp  2024      1    3  100          0
4  Total_Tower2_Adjusted_100KWp  2024      1    4  100          0

Shape of final dataframe: (600, 6)


In [ ]:
import pandas as pd
file_path = '/content/Solar_Generation_Outputs_2022.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)
data_frames = []
current_installation = None
headers = ['Year', 'Hour', 'Size', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
last_processed_index = -1


for index, row in df.iterrows():

    if index <= last_processed_index:
        continue
    if 'Adjusted' in str(row[0]):
        current_installation = row[0]
        continue
    if row[0] == 'Year' and row[1] == 'Hour':
        column_names = row.values
        data_start = index + 1
        continue

    if current_installation and index >= data_start:

        if pd.isna(row[0]):
            continue


        data_end = index
        while data_end < len(df) and not pd.isna(df.iloc[data_end, 0]):
            data_end += 1
        installation_data = df.iloc[index:data_end].copy()
        installation_data.columns = column_names
        installation_data['Installation'] = current_installation
        if not installation_data.empty:
            data_frames.append(installation_data)
        last_processed_index = data_end - 1
if data_frames:
    combined_df = pd.concat(data_frames).drop_duplicates()
else:
    raise ValueError("No valid data found in the Excel file")
melted = pd.melt(combined_df,
                id_vars=['Installation', 'Year', 'Hour', 'Size'],
                value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                var_name='Month',
                value_name='Generation')
month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
             'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
melted['Month'] = melted['Month'].map(month_map)
final_df = melted[['Installation', 'Year', 'Month', 'Hour', 'Size', 'Generation']]
final_df = final_df.drop_duplicates().reset_index(drop=True)

duplicate_check = final_df.duplicated(subset=['Installation', 'Year', 'Month', 'Hour']).sum()
if duplicate_check > 0:
    print(f"Warning: {duplicate_check} duplicates found after processing")
    final_df = final_df.drop_duplicates(subset=['Installation', 'Year', 'Month', 'Hour'])
output_path = '/content/Solar_Generation_Outputs_2022_Transformed.xlsx'
final_df.to_excel(output_path, index=False)

print("Data transformation complete. Saved to:", output_path)
print(f"Final dataset contains {len(final_df)} rows")
print("Sample data:")
final_df.head()